In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))
import time
import math
import re

import cv2
import glob
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.cm as cm
from PIL import Image as Im

from image.image import Image
from control.control import Control
from control import control_funcs as c_funcs
from utils import utils

%matplotlib inline
plt.figure(figsize=(40,40))

%load_ext autoreload
%autoreload 2

In [6]:
# This currently works by importing from the code base, to make it work with the code bellow
# move this cell all the way to the bottom and rerun all cells by click Kernel at the top then
# selecting 'Restart and Run All'

# Create a control for the images and there assocaited data to be
# stored in
basepath = 'set/the/path/to/images'

c = Control('abrasion014_abraded.jpg', 
            'abrasion014_after1.jpg',
            basepath = '/Users/adampaquette/notebooks/Abrasion Images/')

# Add a second image pair
c.add_image_pair('abrasion020_abraded.jpg', 
                 'abrasion020_after2.jpg',
                 basepath = '/Users/adampaquette/notebooks/Abrasion Images/')

# Add a second image pair
c.add_image_pair('abrasion014_marked.png', 
                 'abrasion014_cropped.png',
                 basepath = '/Users/adampaquette/notebooks/Abrasion Images/RockE/')

# Add a second image pair
c.add_image_pair('abrasion020_marked.png', 
                 'abrasion020_cropped.png',
                 basepath = '/Users/adampaquette/notebooks/Abrasion Images/RockE/')

# Apply normalization to each image pair
c.apply_func(c_funcs.normalize_func)

# Create a lab image for the images in the data frame
c.apply_func(c_funcs.convert_func, from_image = 'orig_image_data', to_image = 'lab_image_data', 
             conversion_type = cv2.COLOR_BGR2LAB)

c.apply_func(c_funcs.convert_func, from_image = 'orig_image_data', to_image = 'rgb_image_data', 
             conversion_type = cv2.COLOR_BGR2RGB)

# Apply the subtract function to each pair and generate a subtraction mask
c.apply_func(func = c_funcs.subtract_func, data = 'orig_image_data',useNorm = True)

c.apply_func(func = c_funcs.color_segment_func, rock_type = 'RockE')

In [ ]:
step_size = 1
c.apply_func(func = c_funcs.analyze_mask_func, mask = 'color_mask', mask_type = 'block', buffer = 5, step = step_size)

In [ ]:
dataframe_idx = 2
before_image = c.dataframe.iloc[dataframe_idx ]['before_image']
after_image = c.dataframe.iloc[dataframe_idx ]['after_image']
plt.imshow(after_image['analyzed_image'], cmap = "RdYlGn", interpolation = 'nearest')